## 多板块轮动（六）

In [ ]:
#=====================================================================================================
#=====================================================================================================
# 多板块轮动（六）
#导包操作
import pandas as pd
#=====================================================================================================
#=====================================================================================================
#初始化函数，用于初始条件，其中爱吃醋 context.security中确定策略判断的几大板块
def init(context):   
    run_monthly(trade,date_rule=3)
    context.security=['150019.SZ','159901.OF','510050.OF',
                      '159915.OF','510300.OF',
                      '510500.OF','510180.OF',
                      '159902.OF','159905.OF']
                      #注意：格式可以一行，也可以多行，不影响代码运行，4行还只是context.security里的。
#=====================================================================================================
#=====================================================================================================
#板块强弱衡量函数
def bankuai(context,bar_dict):  
    df={'security':[],'qd':[]}
    stocks=context.security
    for i in stocks:
        df['security'].append(i)
        price = history(i, ['close'], 25, '1d')
        p20=price['close'].iloc[-20]
        p1=price['close'].iloc[-1]
        m=(p1-p20)/p20
        df['qd'].append(m)
    df = pd.DataFrame(df).sort_values(by ='qd', ascending=False)
    x=df['qd'].iloc[0]
    y=['123']
    if x >0 :
        return df['security'].iloc[0]
    else:
        return y
    #直接输出最强的板块，如果所有板块都是负数，那么输出固定值y=['123']
#=====================================================================================================
#=====================================================================================================
#交易函数
def trade(context,bar_dict):
    n=bankuai(context,bar_dict)
    d=len(n)
    #首先判断所有板块是否变弱，变弱则清仓
    if d==1:
        if len(list(context.portfolio.stock_account.positions.keys())) > 0:
            for stock in list(context.portfolio.stock_account.positions.keys()):
                order_target(stock, 0)
    #其次判断最强板块是否是我们持仓的板块，不是则先卖持仓，后买最强板块。
    else:
        if n not in list(list(context.portfolio.stock_account.positions.keys())):
            for stock in list(context.portfolio.stock_account.positions.keys()):
                order_target(stock, 0)
            if len(list(context.portfolio.stock_account.positions.keys())) < 1:
                order_target_value(n, context.portfolio.stock_account.available_cash)
#=====================================================================================================
#=====================================================================================================
#设置止损止盈
def handle_bar(context,bar_dict):
    n=bankuai(context,bar_dict)
    d=len(n)
    #判断函数，用于判断板块，如果所有板块都弱了,那么n是1，执行清仓。
    if d==1:
        if len(list(context.portfolio.stock_account.positions.keys())) > 0:
            for stock in list(list(context.portfolio.stock_account.positions.keys())):
                order_target(stock, 0)
                
                